In [1]:
import configparser
from datetime import datetime
import os
from os import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1570038823663_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# initialize a spark session
spark = SparkSession \
    .builder \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# get filepath to all json file of song_data in S3 bucket
song_data = "s3a://udacity-dend/song_data/A/B/C/*.json"
df = spark.read.json(song_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# print out the schema in tree format
print("---------- Print out the schema of song dataset in tree format: ----------")
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---------- Print out the schema of song dataset in tree format: ----------
root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

In [5]:
# extract columns to create songs table
# songs table attributes: song_id, title, artist_id, year, duration
songs_table = df.select("song_id", "title", "artist_id", "year", "duration")
# show first 10 rows in songs_table
print(" ---------- Show first 10 rows of songs table ----------")
songs_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of songs table ----------
+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SODREIN12A58A7F2E5|A Whiter Shade Of...|ARLTWXK1187FB5A3F8|   0|326.00771|
|SOCEMJV12A6D4F7667|Giant Steps (Alte...|ARIOZCU1187FB3A3DC|   0|220.44689|
|SOWQTQZ12A58A7B63E|Streets On Fire (...|ARPFHN61187FB575F6|   0|279.97995|
|SOMAPYF12A6D4FEC3E|All Day & All Of ...|AR5S9OB1187B9931E3|   0|156.62975|
|SOAPVNX12AB0187625|I Remember Nights...|AR5T40Y1187B9996C6|1998| 249.3122|
|SOFIUVJ12A8C13C296|Will You Tell Me ...|AR9OEB71187B9A97C6|2005|397.16526|
|SOBHXUU12A6D4F5F14|National Emblem (...|ARBDJHO1252CCFA6FC|   0|188.73424|
|SOQFYBD12AB0182188|               Intro|ARAADXM1187FB3ECDB|1999| 67.63057|
|SOYLILV12A8C136650|                 XXX|ARZJDBC1187FB52056|1984|327.00036|
|SOHTCZS12A6D4FC402|  The Chris

In [6]:
# write songs table to parquet files partitioned by year and artist
out_path_songs = "s3a://fzhang22-data-lake/parquet_files/notebook/songs_table.parquet"
# if path.exists(out_path_songs):
songs_table.write.parquet(path = out_path_songs, 
                          partitionBy = ("year", "artist_id"),
                          mode = "overwrite")
# else:
#     songs_table.write.parquet(path = out_path_songs, 
#                               partitionBy = ("year", "artist_id"),
#                               mode = "append")
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# extract columns to create artists table
# artists table attributes: artist_id, name, location, lattitude, longitude
artists_table = df.select("artist_id", "artist_name", "artist_location", 
                          "artist_latitude", "artist_longitude")
# show first 10 rows in artists_table
print(" ---------- Show first 10 rows of artists table ----------")
artists_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of artists table ----------
+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARLTWXK1187FB5A3F8|         King Curtis|      Fort Worth, TX|       32.74863|       -97.32925|
|ARIOZCU1187FB3A3DC|       JOHN COLTRANE|          Hamlet, NC|           null|            null|
|ARPFHN61187FB575F6|         Lupe Fiasco|         Chicago, IL|       41.88415|       -87.63241|
|AR5S9OB1187B9931E3|         Bullet Boys|     Los Angeles, CA|       34.05349|      -118.24532|
|AR5T40Y1187B9996C6|    The Bear Quartet|       Lulea, Sweden|           null|            null|
|AR9OEB71187B9A97C6|              Faunts|Edmonton, Alberta...|           null|            null|
|ARBDJHO1252CCFA6FC|The Band of HM Ro...|                    |           null

In [9]:
# write artists table to parquet files
out_path_artists = "s3a://fzhang22-data-lake/parquet_files/notebook/artists_table.parquet"
artists_table.write.parquet(path = out_path_artists, mode = "overwrite")

# read parquet file and check the first 10 rows of partitioned parquet dataframes
df_artists_parquet = spark.read.parquet("s3a://fzhang22-data-lake/parquet_files/notebook/artists_table.parquet")
print(" ---------- Show first 10 rows of artists table parquet file ----------")
df_artists_parquet.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of artists table parquet file ----------
+------------------+--------------------+--------------------+---------------+----------------+
|         artist_id|         artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+--------------------+--------------------+---------------+----------------+
|ARAADXM1187FB3ECDB|    Styles Of Beyond|  Woodland Hills, CA|        34.1688|      -118.61092|
|ARZJDBC1187FB52056|        Nasty Savage|    Brandon, Florida|       27.94017|       -82.32547|
|AR5S9OB1187B9931E3|         Bullet Boys|     Los Angeles, CA|       34.05349|      -118.24532|
|ARLTWXK1187FB5A3F8|         King Curtis|      Fort Worth, TX|       32.74863|       -97.32925|
|ARPFHN61187FB575F6|         Lupe Fiasco|         Chicago, IL|       41.88415|       -87.63241|
|AR0IAWL1187B9A96D0|        Danilo Perez|              Panama|         8.4177|       -80.11278|
|AR9OEB71187B9A97C6|              Faunts|Edmonton, Alberta...|  

In [10]:
log_data = "s3a://udacity-dend/log_data/2018/11/*.json"

# read log data file
df = spark.read.json(log_data)

# print out the schema in tree format
print("---------- Print out the schema of log dataset in tree format: ----------")
df.printSchema()

# filter by actions for song plays
df = df.filter(df.page == "NextSong")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---------- Print out the schema of log dataset in tree format: ----------
root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [11]:
# extract columns for users table   
# users attributes: user_id, first_name, last_name, gender, level
users_table = df.select("userId", "firstName", "lastName", "gender", "level").distinct()

# show first 10 rows in users table
print(" ---------- Show first 10 rows of users table ----------")
users_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of users table ----------
+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    57|Katherine|     Gay|     F| free|
|    84|  Shakira|    Hunt|     F| free|
|    22|     Sean|  Wilson|     F| free|
|    52| Theodore|   Smith|     M| free|
|    80|    Tegan|  Levine|     F| paid|
|    15|     Lily|    Koch|     F| paid|
|    37|   Jordan|   Hicks|     F| free|
|    98|   Jordyn|  Powell|     F| free|
|    48|   Marina|  Sutton|     F| free|
|    17| Makinley|   Jones|     F| free|
+------+---------+--------+------+-----+
only showing top 10 rows

In [12]:
# write users table to parquet files
output_data_users = "s3a://fzhang22-data-lake/parquet_files/notebook/users_table.parquet"
# if path.exists(output_data_users):
users_table.write.parquet(path = output_data_users, mode = "overwrite")
# else:
#     users_table.write.parquet(path = output_data_users, 
#                               mode = "append")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# read parquet file and check the first 10 rows of partitioned parquet dataframes
df_users_parquet = spark.read.parquet(output_data_users)
print(" ---------- Show first 10 rows of users table parquet file ----------")
df_users_parquet.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of users table parquet file ----------
+------+----------+---------+------+-----+
|userId| firstName| lastName|gender|level|
+------+----------+---------+------+-----+
|    88|  Mohammad|Rodriguez|     M| paid|
|    88|  Mohammad|Rodriguez|     M| free|
|    68|    Jordan|Rodriguez|     F| free|
|    29|Jacqueline|    Lynch|     F| free|
|    11| Christian|   Porter|     F| free|
|    53|   Celeste| Williams|     F| free|
|    69|  Anabelle|  Simpson|     F| free|
|    75|    Joseph|Gutierrez|     M| free|
|    40|    Tucker| Garrison|     M| free|
|     2|   Jizelle| Benjamin|     F| free|
+------+----------+---------+------+-----+
only showing top 10 rows

In [14]:
# create datetime column from original timestamp column
# divide timestamp by 1000 to convert from milliseconds to seconds
get_datetime = udf(lambda x: datetime.fromtimestamp(x / 1000), TimestampType())
df = df.withColumn("start_time", get_datetime(df.ts))

# time table attributes: start_time, hour, day, week, month, year, weekday
get_hour = udf(lambda x: x.hour)  
df = df.withColumn("hour", get_hour(df.start_time))   # create hour column

get_day = udf(lambda x: x.day)
df = df.withColumn("day",  get_day(df.start_time))  # create day column

get_week = udf(lambda x: x.isocalendar()[1])
df = df.withColumn("week", get_week(df.start_time))   # create week number column

get_month = udf(lambda x: x.month)
df = df.withColumn("month", get_month(df.start_time))  # create month column

get_year = udf(lambda x: x.year)
df = df.withColumn("year", get_year(df.start_time)) # create year column

get_weekday = udf(lambda x: x.weekday())
df = df.withColumn("weekday", get_weekday(df.start_time))  # create weekday column

 # extract columns to create time table
time_table = df.select(df.columns[-7:])

# show first 10 rows of time table 
print(" ---------- Show first 10 rows of time table  ----------")
time_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of time table  ----------
+--------------------+----+---+----+-----+----+-------+
|          start_time|hour|day|week|month|year|weekday|
+--------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:...|   0| 15|  46|   11|2018|      3|
|2018-11-15 00:41:...|   0| 15|  46|   11|2018|      3|
|2018-11-15 00:45:...|   0| 15|  46|   11|2018|      3|
|2018-11-15 03:44:...|   3| 15|  46|   11|2018|      3|
|2018-11-15 05:48:...|   5| 15|  46|   11|2018|      3|
|2018-11-15 05:53:...|   5| 15|  46|   11|2018|      3|
|2018-11-15 05:55:...|   5| 15|  46|   11|2018|      3|
|2018-11-15 06:01:...|   6| 15|  46|   11|2018|      3|
|2018-11-15 06:07:...|   6| 15|  46|   11|2018|      3|
|2018-11-15 06:10:...|   6| 15|  46|   11|2018|      3|
+--------------------+----+---+----+-----+----+-------+
only showing top 10 rows

In [15]:
# write time table to parquet files partitioned by year and month
out_path_time = "s3a://fzhang22-data-lake/parquet_files/notebook/time_table.parquet"
# if path.exists(out_path_time):
time_table.write.parquet(path = out_path_time, 
                         partitionBy = ("year", "month"),
                         mode = "overwrite")
# else:
#     time_table.write.parquet(path = out_path_time, 
#                              partitionBy = ("year", "month"),
#                              mode = "append")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# read parquet file and check the first 10 rows of partitioned parquet dataframes
df_time_parquet = spark.read.parquet(out_path_time)
print(" ---------- Show first 10 rows of time table parquet file ----------")
df_time_parquet.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of time table parquet file ----------
+--------------------+----+---+----+-------+----+-----+
|          start_time|hour|day|week|weekday|year|month|
+--------------------+----+---+----+-------+----+-----+
|2018-11-15 00:30:...|   0| 15|  46|      3|2018|   11|
|2018-11-15 00:41:...|   0| 15|  46|      3|2018|   11|
|2018-11-15 00:45:...|   0| 15|  46|      3|2018|   11|
|2018-11-15 03:44:...|   3| 15|  46|      3|2018|   11|
|2018-11-15 05:48:...|   5| 15|  46|      3|2018|   11|
|2018-11-15 05:53:...|   5| 15|  46|      3|2018|   11|
|2018-11-15 05:55:...|   5| 15|  46|      3|2018|   11|
|2018-11-15 06:01:...|   6| 15|  46|      3|2018|   11|
|2018-11-15 06:07:...|   6| 15|  46|      3|2018|   11|
|2018-11-15 06:10:...|   6| 15|  46|      3|2018|   11|
+--------------------+----+---+----+-------+----+-----+
only showing top 10 rows

In [17]:
# read in song data to use for songplays table
song_df = spark.read.parquet(out_path_songs)

# inner join df with song_df by song's name
cond = [df.song == song_df.title]
df_join = df.join(song_df, cond, "inner")

# extract columns from joined song and log datasets to create songplays table 
# songplays attributes: songplay_id, start_time, user_id, level, song_id, 
#                       artist_id, session_id, location, user_agent
songplays_table = df_join.select("start_time", "userId", "level", "song_id", 
                                 "artist_id", "sessionId", "location", 
                                 "userAgent").distinct()

# create songplay_id column with auto_increment
songplays_table.withColumn("songplay_id", monotonically_increasing_id())

# show first 10 rows of songplays table 
print(" ---------- Show first 10 rows of songplays table  ----------")
songplays_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of songplays table  ----------
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|          start_time|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+
|2018-11-27 22:35:...|    80| paid|SOQFYBD12AB0182188|ARAADXM1187FB3ECDB|      992|Portland-South Po...|"Mozilla/5.0 (Mac...|
|2018-11-14 05:06:...|    10| free|SOGDBUF12A8C140FAA|AR558FS1187FB45658|      484|Washington-Arling...|"Mozilla/5.0 (Mac...|
|2018-11-19 09:14:...|    24| paid|SOQFYBD12AB0182188|ARAADXM1187FB3ECDB|      672|Lake Havasu City-...|"Mozilla/5.0 (Win...|
|2018-11-19 09:14:...|    24| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|      672|Lake Havasu City-...|"Mozilla/5.0 (Win...|
|2018-11-14 05:06:...|    10| free|SOQFYBD12AB0182188|AR

In [24]:
songplays_table = songplays_table.withColumn("year", get_year(df.start_time))
songplays_table = songplays_table.withColumn("month", get_month(df.start_time))

# write songplays table to parquet files partitioned by year and month
out_path_songplays = "s3a://fzhang22-data-lake/parquet_files/notebook/songplays_table.parquet"
# if path.exists(out_path_songplays):
songplays_table.write.parquet(path = out_path_songplays, 
                         partitionBy = ("year", "month"),
                         mode = "overwrite")
# else:
#     time_table.write.parquet(path = out_path_songplays, 
#                              partitionBy = ("year", "month"),
#                              mode = "append")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# show first 10 rows of songplays table 
print(" ---------- Show first 10 rows of songplays table  ----------")
songplays_table.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 ---------- Show first 10 rows of songplays table  ----------
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+
|          start_time|userId|level|           song_id|         artist_id|sessionId|            location|           userAgent|year|month|
+--------------------+------+-----+------------------+------------------+---------+--------------------+--------------------+----+-----+
|2018-11-27 22:35:...|    80| paid|SOQFYBD12AB0182188|ARAADXM1187FB3ECDB|      992|Portland-South Po...|"Mozilla/5.0 (Mac...|2018|   11|
|2018-11-14 05:06:...|    10| free|SOGDBUF12A8C140FAA|AR558FS1187FB45658|      484|Washington-Arling...|"Mozilla/5.0 (Mac...|2018|   11|
|2018-11-19 09:14:...|    24| paid|SOQFYBD12AB0182188|ARAADXM1187FB3ECDB|      672|Lake Havasu City-...|"Mozilla/5.0 (Win...|2018|   11|
|2018-11-19 09:14:...|    24| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|      672|Lake Havasu City-...|"Mozilla/5.0